In [26]:
import findspark
findspark.init()
findspark.find()

import os

# Check JAVA_HOME
java_home = os.environ.get('JAVA_HOME')
print(f'JAVA_HOME: {java_home}')

# Check SPARK_HOME
spark_home = os.environ.get('SPARK_HOME')
print(f'SPARK_HOME: {spark_home}')

# Verify Java version
java_version = os.popen('java -version 2>&1').read().strip()
print(f'Java version: {java_version}')

# Verify Spark version
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CheckVersions").getOrCreate()
spark_version = spark.version
print(f'Spark version: {spark_version}')

spark.stop()

JAVA_HOME: C:\Program Files\Java\jdk-17
SPARK_HOME: C:\spark\spark-3.5.1-bin-hadoop3
Java version: java version "17.0.11" 2024-04-16 LTS
Java(TM) SE Runtime Environment (build 17.0.11+7-LTS-207)
Java HotSpot(TM) 64-Bit Server VM (build 17.0.11+7-LTS-207, mixed mode, sharing)
Spark version: 3.5.1


In [134]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, levenshtein ,when, greatest ,length , lit , lower

In [28]:
spark = SparkSession.builder.appName("PercentageMatchingExample").getOrCreate()


In [29]:
syn=spark.read.csv(r"C:\Users\sansk\Downloads\pyspark\Synthetic_kyc_data.csv",header=True,inferSchema=True,sep=",",encoding="UTF-8",nullValue="NULL")

In [30]:
syn.show()

+-----------+---------------+----------+----------+-------------+-----------+------+--------------+-----------------+--------------------+-------------+-----------+--------------+------------+--------------+--------------------+--------------------+
|Customer Id|Passport Number|First Name| Last Name|Date of Birth|Blood Group|Gender|Marital Status|  Education Level|             Address|         City|Postal Code|       Country|Country Code|  Phone Number|        Company Name|               Email|
+-----------+---------------+----------+----------+-------------+-----------+------+--------------+-----------------+--------------------+-------------+-----------+--------------+------------+--------------+--------------------+--------------------+
| 3728211145|35SEJI84WPXDGBF|     Lacey|   Charles|   1958-09-05|         O+|Female|       widowed|      High School|67969 Eddie Land,...|San Francisco|      19222| United States|          US| +1-3276914085|Walton  Decker an...|lacey.charles@wal...|


In [31]:
equ=spark.read.csv(r"C:\Users\sansk\Downloads\pyspark\equifax_third_party_synthetic_banking_kyc_data.csv",header=True,inferSchema=True,sep=",",encoding="UTF-8",nullValue="NULL")

In [32]:
equ.show()


+---------------+----------+----------+-------------+-----------+------+--------------+-----------------+--------------------+------------+-----------+--------------+------------+--------------+--------------------+--------------------+
|Passport Number|First Name| Last Name|Date of Birth|Blood Group|Gender|Marital Status|  Education Level|             Address|        City|Postal Code|       Country|Country Code|  Phone Number|        Company Name|               Email|
+---------------+----------+----------+-------------+-----------+------+--------------+-----------------+--------------------+------------+-----------+--------------+------------+--------------+--------------------+--------------------+
|35SEJI84WPXDGBF|     Lacey|   Charles|   1958-09-05|         O+|Female|       widowed|      High School|067 Decker Missio...|      Austin|      19222| United States|          US| +1-3276914085|Walton  Decker an...|lacey.charles@wal...|
|YPMLQ1AKG2T6H8R|   Anthony|      Hart|     60-07-08

In [33]:
dnb=spark.read.csv(r"C:\Users\sansk\Downloads\pyspark\dnb_third_party_synthetic_banking_kyc_data.csv",header=True,inferSchema=True,sep=",",encoding="UTF-8",nullValue="NULL")

In [34]:
dnb.show()

+---------------+----------+----------+-------------+-----------+------+--------------+-----------------+--------------------+-------------+-----------+--------------+------------+--------------+--------------------+--------------------+
|Passport Number|First Name| Last Name|Date of Birth|Blood Group|Gender|Marital Status|  Education Level|             Address|         City|Postal Code|       Country|Country Code|  Phone Number|        Company Name|               Email|
+---------------+----------+----------+-------------+-----------+------+--------------+-----------------+--------------------+-------------+-----------+--------------+------------+--------------+--------------------+--------------------+
|35SEJI84WPXDGBF|     Lacey|   Charles|   1958-09-05|         O+|Female|       widowed|      High School|67969 Eddie Land,...|San Francisco|      19222| United States|          US| +1-3276914085|Walton  Decker an...|lacey.charles@wal...|
|YPMLQ1AKG2T6H8R|   Anthony|      Hart|   1960-0

In [35]:
equ.printSchema()


root
 |-- Passport Number: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Date of Birth: string (nullable = true)
 |-- Blood Group: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Education Level: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Postal Code: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Country Code: string (nullable = true)
 |-- Phone Number: string (nullable = true)
 |-- Company Name: string (nullable = true)
 |-- Email: string (nullable = true)



In [36]:
equ.printSchema()

root
 |-- Passport Number: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Date of Birth: string (nullable = true)
 |-- Blood Group: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Education Level: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Postal Code: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Country Code: string (nullable = true)
 |-- Phone Number: string (nullable = true)
 |-- Company Name: string (nullable = true)
 |-- Email: string (nullable = true)



In [46]:
equ=spark.read.csv(r"C:\Users\sansk\Downloads\pyspark\equifax_third_party_synthetic_banking_kyc_data.csv",header=True,inferSchema=True,sep=",",encoding="UTF-8",nullValue="NULL")

In [47]:
equ.printSchema()

root
 |-- Passport Number: string (nullable = true)
 |-- equ_First_Name: string (nullable = true)
 |-- equ_Last_Name: string (nullable = true)
 |-- equ_Date_of_Birth: string (nullable = true)
 |-- equ_Blood_Group: string (nullable = true)
 |-- equ_Gender: string (nullable = true)
 |-- equ_Marital_Status: string (nullable = true)
 |-- equ_Education_Level: string (nullable = true)
 |-- equ_Address: string (nullable = true)
 |-- equ_City: string (nullable = true)
 |-- equ_Postal_Code: string (nullable = true)
 |-- equ_Country: string (nullable = true)
 |-- equ_Country_Code: string (nullable = true)
 |-- equ_Phone_Number: string (nullable = true)
 |-- equ_Company_Name: string (nullable = true)
 |-- equ_Email: string (nullable = true)



In [48]:
dnb=spark.read.csv(r"C:\Users\sansk\Downloads\pyspark\dnb_third_party_synthetic_banking_kyc_data.csv",header=True,inferSchema=True,sep=",",encoding="UTF-8",nullValue="NULL")

In [49]:
dnb.printSchema()

root
 |-- Passport Number: string (nullable = true)
 |-- dnb_First_Name: string (nullable = true)
 |-- dnb_Last_Name: string (nullable = true)
 |-- dnb_Date_of_Birth: string (nullable = true)
 |-- dnb_Blood_Group: string (nullable = true)
 |-- dnb_Gender: string (nullable = true)
 |-- dnb_Marital_Status: string (nullable = true)
 |-- dnb_Education_Level: string (nullable = true)
 |-- dnb_Address: string (nullable = true)
 |-- dnb_City: string (nullable = true)
 |-- dnb_Postal_Code: string (nullable = true)
 |-- dnb_Country: string (nullable = true)
 |-- dnb_Country_Code: string (nullable = true)
 |-- dnb_Phone_Number: string (nullable = true)
 |-- dnb_Company_Name: string (nullable = true)
 |-- dnb_Email: string (nullable = true)



In [22]:
syn.show()

+-----------+---------------+----------+----------+-------------+-----------+------+--------------+-----------------+--------------------+-------------+-----------+--------------+------------+--------------+--------------------+--------------------+
|Customer Id|Passport Number|First Name| Last Name|Date of Birth|Blood Group|Gender|Marital Status|  Education Level|             Address|         City|Postal Code|       Country|Country Code|  Phone Number|        Company Name|               Email|
+-----------+---------------+----------+----------+-------------+-----------+------+--------------+-----------------+--------------------+-------------+-----------+--------------+------------+--------------+--------------------+--------------------+
| 3728211145|35SEJI84WPXDGBF|     Lacey|   Charles|   1958-09-05|         O+|Female|       widowed|      High School|67969 Eddie Land,...|San Francisco|      19222| United States|          US| +1-3276914085|Walton  Decker an...|lacey.charles@wal...|


In [50]:
joined_syn_equ = syn.join(equ,"Passport Number")

In [51]:
dr = joined_syn_equ.join(dnb,"Passport Number")

In [52]:
dr.show()


+---------------+-----------+-----------+---------+-------------+-----------+------+--------------+-----------------+--------------------+-------------+-----------+--------------+------------+--------------+--------------------+--------------------+--------------+-------------+-----------------+---------------+----------+------------------+-------------------+--------------------+-------------+---------------+--------------+----------------+----------------+--------------------+--------------------+--------------+-------------+-----------------+---------------+----------+------------------+-------------------+--------------------+-------------+---------------+--------------+----------------+----------------+--------------------+--------------------+
|Passport Number|Customer Id| First Name|Last Name|Date of Birth|Blood Group|Gender|Marital Status|  Education Level|             Address|         City|Postal Code|       Country|Country Code|  Phone Number|        Company Name|            

In [53]:
dr.printSchema()

root
 |-- Passport Number: string (nullable = true)
 |-- Customer Id: long (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Date of Birth: date (nullable = true)
 |-- Blood Group: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Education Level: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Postal Code: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Country Code: string (nullable = true)
 |-- Phone Number: string (nullable = true)
 |-- Company Name: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- equ_First_Name: string (nullable = true)
 |-- equ_Last_Name: string (nullable = true)
 |-- equ_Date_of_Birth: string (nullable = true)
 |-- equ_Blood_Group: string (nullable = true)
 |-- equ_Gender: string (nullable = true)
 |-- equ_Marital_Status: string (nullable = true)
 |

In [54]:
dr = dr.withColumn("max_length_First_Name_equ", when(col("First Name").isNotNull() & col("equ_First_Name").isNotNull(),
                                       greatest(length(col("First Name")), length(col("equ_First_Name"))))
                                      .otherwise(lit(0)))

In [55]:
dr = dr.withColumn("levenshtein_distance_First_Name_equ", levenshtein(col("First Name"), col("equ_First_Name")))

In [57]:
dr = dr.withColumn("equ_match_FN", 
                   (1 - (col("levenshtein_distance_First_Name_equ") / col("max_length_First_Name_equ"))) * 100)

In [58]:
dr.select("First Name", "equ_First_Name", "equ_match_FN").show(truncate=False)

+----------+--------------+------------+
|First Name|equ_First_Name|equ_match_FN|
+----------+--------------+------------+
|Lacey     |Lacey         |100.0       |
|Anthony   |Anthony       |100.0       |
|Dean      |Dean          |100.0       |
|David     |David         |100.0       |
|Elizabeth |Elizabeth     |100.0       |
|Dawn      |Dawn          |100.0       |
|Paul      |Paul          |100.0       |
|Janice    |Janice        |100.0       |
|Douglas   |Douglas       |100.0       |
|Mary      |Mary          |100.0       |
|Jodie     |Jodie         |100.0       |
|Mark      |Mark          |100.0       |
|Charles   |Charles       |100.0       |
|Declan    |Declan        |100.0       |
|Robert    |Robert        |100.0       |
|David     |David         |100.0       |
|Colin     |Colin         |100.0       |
|Dean      |Dean          |100.0       |
|Lauren    |Lauren        |100.0       |
|Rosemary  |Rosemary      |100.0       |
+----------+--------------+------------+
only showing top

In [63]:
syn.select("Blood Group").distinct().show()


+-----------+
|Blood Group|
+-----------+
|        AB+|
|         O+|
|        AB-|
|         A-|
|         O-|
|         B+|
|         A+|
|         B-|
+-----------+



In [76]:
equ.select("equ_Blood_Group").distinct().show()

+---------------+
|equ_Blood_Group|
+---------------+
|            AB+|
|             O+|
|            AB-|
|             A-|
|             O-|
|             B+|
|             A+|
|             B-|
+---------------+



In [71]:
equ=equ.replace("A Positive","A+",subset=["equ_Blood_Group"])

In [73]:
equ=equ.replace("A Negative","A-",subset=["equ_Blood_Group"])

In [74]:
equ=equ.replace("B Positive","B+",subset=["equ_Blood_Group"])

In [75]:
equ=equ.replace("B Negative","B-",subset=["equ_Blood_Group"])

In [84]:
dnb.select("dnb_Blood_Group").distinct().show()

+---------------+
|dnb_Blood_Group|
+---------------+
|            AB+|
|             O+|
|            AB-|
|             A-|
|             O-|
|             B+|
|             A+|
|             B-|
+---------------+



In [80]:
dnb=dnb.replace("A Positive","A+",subset=["dnb_Blood_Group"])

In [81]:
dnb=dnb.replace("B Positive","B+",subset=["dnb_Blood_Group"])

In [82]:
dnb=dnb.replace("A Negative","A-",subset=["dnb_Blood_Group"])

In [83]:
dnb=dnb.replace("B Negative","B-",subset=["dnb_Blood_Group"])

In [91]:
def fix_date_format(df):
    for col_name in df.columns:
        if col_name == "Date of Birth":
            # Check if the column contains dates in "yy-mm-dd" format
            if df.select(col(col_name).cast("string").rlike(r"^\d{2}-\d{2}-\d{2}$")).count() > 0:
                # Add "19" to the beginning of the date strings
                df = df.withColumn(col_name, 
                                   when(col(col_name).cast("string").rlike(r"^\d{2}-\d{2}-\d{2}$"), 
                                        19 + col(col_name))
                                  .otherwise(col(col_name)))
    return df

In [92]:
fix_date_format(syn)

DataFrame[Customer Id: bigint, Passport Number: string, First Name: string, Last Name: string, Date of Birth: date, Blood Group: string, Gender: string, Marital Status: string, Education Level: string, Address: string, City: string, Postal Code: string, Country: string, Country Code: string, Phone Number: string, Company Name: string, Email: string]

In [93]:
syn.show()

+-----------+---------------+----------+----------+-------------+-----------+------+--------------+-----------------+--------------------+-------------+-----------+--------------+------------+--------------+--------------------+--------------------+
|Customer Id|Passport Number|First Name| Last Name|Date of Birth|Blood Group|Gender|Marital Status|  Education Level|             Address|         City|Postal Code|       Country|Country Code|  Phone Number|        Company Name|               Email|
+-----------+---------------+----------+----------+-------------+-----------+------+--------------+-----------------+--------------------+-------------+-----------+--------------+------------+--------------+--------------------+--------------------+
| 3728211145|35SEJI84WPXDGBF|     Lacey|   Charles|   1958-09-05|         O+|Female|       widowed|      High School|67969 Eddie Land,...|San Francisco|      19222| United States|          US| +1-3276914085|Walton  Decker an...|lacey.charles@wal...|


In [104]:
dnb.select("dnb_Gender").distinct().show()


+----------+
|dnb_Gender|
+----------+
|    Female|
|      Male|
+----------+



In [102]:
dnb=dnb.replace("M","Male",subset=["dnb_Gender"])

In [103]:
dnb=dnb.replace("F","Female",subset=["dnb_Gender"])

In [105]:
fix_date_format(equ)

DataFrame[Passport Number: string, equ_First_Name: string, equ_Last_Name: string, equ_Date_of_Birth: string, equ_Blood_Group: string, equ_Gender: string, equ_Marital_Status: string, equ_Education_Level: string, equ_Address: string, equ_City: string, equ_Postal_Code: string, equ_Country: string, equ_Country_Code: string, equ_Phone_Number: string, equ_Company_Name: string, equ_Email: string]

In [106]:
fix_date_format(dnb)

DataFrame[Passport Number: string, dnb_First_Name: string, dnb_Last_Name: string, dnb_Date_of_Birth: string, dnb_Blood_Group: string, dnb_Gender: string, dnb_Marital_Status: string, dnb_Education_Level: string, dnb_Address: string, dnb_City: string, dnb_Postal_Code: string, dnb_Country: string, dnb_Country_Code: string, dnb_Phone_Number: string, dnb_Company_Name: string, dnb_Email: string]

+------------------+
|equ_Marital_Status|
+------------------+
|          divorced|
|           married|
|            single|
|           widowed|
+------------------+



In [126]:


def compare_columns(df, column1, column2):
    # Generate a unique name for the comparison result column
    comparison_name = f"match%_{column2}"

    # Calculate the maximum length of the two columns
    max_length_column1_column2 = df.withColumn(comparison_name + "_max_length",
                                               greatest(length(col(column1)), length(col(column2))))

    # Calculate the Levenshtein distance between the two columns
    df_with_levenshtein = max_length_column1_column2.withColumn(comparison_name + "_levenshtein_distance",
                                                                levenshtein(col(column1), col(column2)))

    # Calculate the percentage match
    df_with_percentage_match = df_with_levenshtein.withColumn(comparison_name + "_match_percentage",
                                                              (1 - (col(comparison_name + "_levenshtein_distance") /
                                                                     col(comparison_name + "_max_length"))) * 100)
    

    return df_with_percentage_match

# Example usage:
# df = your DataFrame
# new_df = compare_columns(df, "First Name", "equ_First_Name")
# new_df.show()


In [138]:
dr = compare_columns(dr, "First Name", "dnb_First_Name")
dr.printSchema()

root
 |-- Passport Number: string (nullable = true)
 |-- Customer Id: long (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Date of Birth: date (nullable = true)
 |-- Blood Group: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Education Level: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Postal Code: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Country Code: string (nullable = true)
 |-- Phone Number: string (nullable = true)
 |-- Company Name: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- equ_First_Name: string (nullable = true)
 |-- equ_Last_Name: string (nullable = true)
 |-- equ_Date_of_Birth: string (nullable = true)
 |-- equ_Blood_Group: string (nullable = true)
 |-- equ_Gender: string (nullable = true)
 |-- equ_Marital_Status: string (nullable = true)
 |

In [139]:
dr.select("First Name", "equ_First_Name","dnb_First_Name","match%_equ_First_Name_match_percentage","match%_dnb_First_Name_match_percentage").show()

+----------+--------------+--------------+--------------------------------------+--------------------------------------+
|First Name|equ_First_Name|dnb_First_Name|match%_equ_First_Name_match_percentage|match%_dnb_First_Name_match_percentage|
+----------+--------------+--------------+--------------------------------------+--------------------------------------+
|     Lacey|         Lacey|         Lacey|                                 100.0|                                 100.0|
|   Anthony|       Anthony|       Anthony|                                 100.0|                                 100.0|
|      Dean|          Dean|          Dean|                                 100.0|                                 100.0|
|     David|         David|         David|                                 100.0|                                 100.0|
| Elizabeth|     Elizabeth|     Elizabeth|                                 100.0|                                 100.0|
|      Dawn|          Dawn|     

In [156]:
dr = compare_columns(dr, "Last Name", "equ_Last_Name")
dr.printSchema()

root
 |-- Passport Number: string (nullable = true)
 |-- Customer Id: long (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Date of Birth: date (nullable = true)
 |-- Blood Group: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Education Level: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Postal Code: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Country Code: string (nullable = true)
 |-- Phone Number: string (nullable = true)
 |-- Company Name: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- equ_First_Name: string (nullable = true)
 |-- equ_Last_Name: string (nullable = true)
 |-- equ_Date_of_Birth: string (nullable = true)
 |-- equ_Blood_Group: string (nullable = true)
 |-- equ_Gender: string (nullable = true)
 |-- equ_Marital_Status: string (nullable = true)
 |

In [227]:
detail_report=dr.select("First Name", "equ_First_Name","dnb_First_Name","match%_equ_First_Name_match_percentage","match%_dnb_First_Name_match_percentage","Last Name", "equ_Last_Name","dnb_Last_Name","match%_equ_Last_Name_match_percentage","match%_dnb_Last_Name_match_percentage","Date of Birth", "equ_Date_of_Birth","dnb_Date_of_Birth","match%_equ_Date_of_Birth_match_percentage","match%_dnb_Date_of_Birth_match_percentage","Blood Group", "equ_Blood_Group","dnb_Blood_Group","match%_equ_Blood_Group_match_percentage","match%_dnb_Blood_Group_match_percentage","Gender", "equ_Gender","dnb_Gender","match%_equ_Gender_match_percentage","match%_dnb_Gender_match_percentage","Marital Status", "equ_Marital_Status","dnb_Marital_Status","match%_equ_Marital_Status_match_percentage","match%_dnb_Marital_Status_match_percentage","Education Level", "equ_Education_Level","dnb_Education_Level","match%_equ_Education_Level_match_percentage","match%_dnb_Education_Level_match_percentage","Address", "equ_Address","dnb_Address","match%_equ_Address_match_percentage","match%_dnb_Address_match_percentage","City", "equ_City", "dnb_City","match%_equ_City_match_percentage","match%_dnb_City_match_percentage","Postal Code", "equ_Postal_Code","dnb_Postal_Code","match%_equ_Postal_Code_match_percentage","match%_dnb_Postal_Code_match_percentage","Country", "equ_Country","dnb_Country","match%_equ_Country_match_percentage","match%_dnb_Country_match_percentage","Country Code", "equ_Country_Code", "dnb_Country_Code","match%_equ_Country_Code_match_percentage","match%_dnb_Country_Code_match_percentage","Phone Number", "equ_Phone_Number","dnb_Phone_Number","match%_equ_Phone_Number_match_percentage","match%_dnb_Phone_Number_match_percentage","Company Name", "equ_Company_Name", "dnb_Company_Name","match%_equ_Company_Name_match_percentage","match%_dnb_Company_Name_match_percentage","Email", "equ_Email", "dnb_Email","match%_equ_Email_match_percentage","match%_dnb_Email_match_percentage")

In [225]:
dr = compare_columns(dr, "Email", "dnb_Email")
dr.printSchema()

root
 |-- Passport Number: string (nullable = true)
 |-- Customer Id: long (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Date of Birth: date (nullable = true)
 |-- Blood Group: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Education Level: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Postal Code: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Country Code: string (nullable = true)
 |-- Phone Number: string (nullable = true)
 |-- Company Name: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- equ_First_Name: string (nullable = true)
 |-- equ_Last_Name: string (nullable = true)
 |-- equ_Date_of_Birth: string (nullable = true)
 |-- equ_Blood_Group: string (nullable = true)
 |-- equ_Gender: string (nullable = true)
 |-- equ_Marital_Status: string (nullable = true)
 |

In [275]:
detail_report.show()
detail_report.printSchema()

+-----------+--------------+--------------+--------------------------------------+--------------------------------------+---------+-------------+-------------+-------------------------------------+-------------------------------------+-------------+-----------------+-----------------+-----------------------------------------+-----------------------------------------+-----------+---------------+---------------+---------------------------------------+---------------------------------------+------+----------+----------+----------------------------------+----------------------------------+--------------+------------------+------------------+------------------------------------------+------------------------------------------+-----------------+-------------------+-------------------+-------------------------------------------+-------------------------------------------+--------------------+--------------------+--------------------+-----------------------------------+------------------------

In [230]:
syn=spark.read.csv(r"C:\Users\sansk\Downloads\pyspark\Synthetic_kyc_data.csv",header=True,inferSchema=True,sep=",",encoding="UTF-8",nullValue="NULL")

In [267]:
sr=spark.read.csv(r"C:\Users\sansk\Downloads\pyspark\sr.csv.txt",header=True,inferSchema=True,sep=",",encoding="UTF-8",nullValue="NULL")

In [268]:
sr.show()

+---------------+----------------------------+----------------------+
|attribute names|overall experian matching % |overall D&B matching %|
+---------------+----------------------------+----------------------+
|passport Number|                       100.0|                 100.0|
|     fisrt name|                       100.0|                 100.0|
|      last name|                       100.0|                 100.0|
|  date of birth|                     95.9788|               96.9862|
|    blood group|                     91.0036|               93.3715|
|         gender|           84.26375000000156|      88.1480833333347|
| marital status|                       100.0|                 100.0|
|education level|           87.92690944741793|     90.96800427807685|
|        address|           85.57572036533742|     89.10726807366501|
|           city|           83.50448777888673|     87.57197336829755|
|    postal code|           82.33951071428505|     86.73370714285663|
|        country|   

In [234]:
detail_report.printSchema()

root
 |-- First Name: string (nullable = true)
 |-- equ_First_Name: string (nullable = true)
 |-- dnb_First_Name: string (nullable = true)
 |-- match%_equ_First_Name_match_percentage: double (nullable = true)
 |-- match%_dnb_First_Name_match_percentage: double (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- equ_Last_Name: string (nullable = true)
 |-- dnb_Last_Name: string (nullable = true)
 |-- match%_equ_Last_Name_match_percentage: double (nullable = true)
 |-- match%_dnb_Last_Name_match_percentage: double (nullable = true)
 |-- Date of Birth: date (nullable = true)
 |-- equ_Date_of_Birth: string (nullable = true)
 |-- dnb_Date_of_Birth: string (nullable = true)
 |-- match%_equ_Date_of_Birth_match_percentage: double (nullable = true)
 |-- match%_dnb_Date_of_Birth_match_percentage: double (nullable = true)
 |-- Blood Group: string (nullable = true)
 |-- equ_Blood_Group: string (nullable = true)
 |-- dnb_Blood_Group: string (nullable = true)
 |-- match%_equ_Blood_Group_

In [233]:
from pyspark.sql.functions import avg

def calculate_column_average(df, column_name):
    # Calculate the average of the specified column
    column_avg = df.select(avg(column_name)).collect()[0][0]
    
    return column_avg

In [271]:
calculate_column_average(detail_report,"match%_dnb_Email_match_percentage")

100.0

In [269]:
detail_report.show()


+-----------+--------------+--------------+--------------------------------------+--------------------------------------+---------+-------------+-------------+-------------------------------------+-------------------------------------+-------------+-----------------+-----------------+-----------------------------------------+-----------------------------------------+-----------+---------------+---------------+---------------------------------------+---------------------------------------+------+----------+----------+----------------------------------+----------------------------------+--------------+------------------+------------------+------------------------------------------+------------------------------------------+-----------------+-------------------+-------------------+-------------------------------------------+-------------------------------------------+--------------------+--------------------+--------------------+-----------------------------------+------------------------

In [270]:
sr.show()


+---------------+----------------------------+----------------------+
|attribute names|overall experian matching % |overall D&B matching %|
+---------------+----------------------------+----------------------+
|passport Number|                       100.0|                 100.0|
|     fisrt name|                       100.0|                 100.0|
|      last name|                       100.0|                 100.0|
|  date of birth|                     95.9788|               96.9862|
|    blood group|                     91.0036|               93.3715|
|         gender|           84.26375000000156|      88.1480833333347|
| marital status|                       100.0|                 100.0|
|education level|           87.92690944741793|     90.96800427807685|
|        address|           85.57572036533742|     89.10726807366501|
|           city|           83.50448777888673|     87.57197336829755|
|    postal code|           82.33951071428505|     86.73370714285663|
|        country|   

In [272]:
calculate_column_average(sr,"overall experian matching % ")

93.58006993532932

In [273]:
calculate_column_average(sr,"overall D&B matching %")

95.17223591611055

In [ ]:
#D&B has been performing better 